In [1]:
%reload_ext jupyter_widget_hwt

# HWT essentials
# Signal is a basic interface, its is like wire in Verilog or port in VHDL
from hwt.interfaces.std import Signal
# Unit is a base class of HWT components
from hwt.synthesizer.unit import Unit

# jupyter_code_format is just shortcut cunction for pygments syntax highligh
from jupyter_code_format import jupyter_code_format


# This is a very simple component which is just a direct wire
class DirectWire(Unit):
    """
    In order to create a new unit you have to make new class derived from Unit (or subunit).
    """

    def _declr(self):
        """
        _declr() is like header of Verilog module or VHDL entity.
        There you have to declare things which should be visible from outside.
        """
        # interfaces "a" and "b" are accessible from outside when declared
        # in _declr method, this means they will be interfaces of Entity/Module
        # and parent unit can connect signals to this interfaces
        # note that interfaces has to be properties of this object
        # which is kind of registration and without it, they can not be discovered
        self.a = Signal()
        # "b" is output and has to be marked with "._m()" as master then
        self.b = Signal()._m()

    def _impl(self):
        """
        _impl() is like body of Verilog module or VHDL architecture.
        Logic and connections are specified i`qn this function.
        """

        # call operator creates assignment. First parameter is source rest
        # are destinations.
        self.b(self.a)  # a drives b, "()" because "=" can not be overriden

In [2]:
# You can use HwtSchemeWidget to draw the component schematic (if you have it installed)
from jupyter_widget_hwt import HwtSchemeWidget
HwtSchemeWidget(DirectWire(), width=200, height=40)

HwtSchemeWidget(height='40px', value={'hwMeta': {'name': 'DirectWire', 'maxId': 5}, 'properties': {'org.eclips…

In [3]:
# to_rtl_str can be imported anywhere but we prefer to import it only
# when this script is running as main
from hwt.synthesizer.utils import to_rtl_str
from hwt.serializer.vhdl import Vhdl2008Serializer


# we create instance of our unit
u = DirectWire()
# to_rtl_str() returns hdl string, you can also generate files with to_rtl, IP-xact packages with IpPackager
vhdl_str = to_rtl_str(u, serializer_cls=Vhdl2008Serializer)
jupyter_code_format(vhdl_str, 'vhdl')

In [4]:
from hwt.serializer.verilog import VerilogSerializer
verilog_str = to_rtl_str(DirectWire(), serializer_cls=VerilogSerializer)
jupyter_code_format(verilog_str, 'verilog')

In [5]:
# you can use IpPackager to export the component as IP-XACT format
# to import componet to Xilinx Vivado/Intel Quartus
from hwt.serializer.ip_packager import IpPackager
p = IpPackager(DirectWire(), name="simple_ip")
p.createPackage("/tmp/")

!ls /tmp/simple_ip

component_hw.tcl  component.xml  src  xgui
